In [3]:
import mysql.connector
from mysql.connector import Error

In [5]:
from dotenv import load_dotenv
import os


In [6]:
load_dotenv()

True

In [4]:
hostname=os.getenv("hostname")
username=os.getenv("username")
password=os.getenv("password")
database=os.getenv("database")
port=os.getenv("port")

try:
    connection=mysql.connector.connect(
        host=hostname,
        user=username,
        password=password,
        database=database,
        port=port
    )
    if connection.is_connected():
        db_info=connection.get_server_info()
        print("Connected to MySQL Server version:", db_info)
        cursor= connection.cursor()
        cursor.execute("SELECT DATABASE();")
        current_database = cursor.fetchone()
        print("Current database:", current_database)
except Error as e:
    print("Error while connecting to MySQL:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

C:\Users\visio\AppData\Local\Temp\ipykernel_37396\964179517.py:16: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info=connection.get_server_info()


Connected to MySQL Server version: 8.0.36-28
Current database: ('olistsql_drewteaget',)
MySQL connection is closed


In [12]:
import pandas as pd

order_payments = pd.read_csv("Data/olist_order_payments_dataset.csv")
order_payments.head()
order_payments.shape


(103886, 5)

In [16]:
csv_file_path = "Data/olist_order_payments_dataset.csv"
table_name = "order_payments"

try:
    connection = mysql.connector.connect(
        host=hostname,
        user=username,
        password=password,
        database=database,
        port=port
    )
    if connection.is_connected():
        print("Connected to MySQL Server", connection.get_server_info())
        cursor = connection.cursor()
        
        # Create table if it does not exist
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            order_id VARCHAR(50),
            payment_sequential INT,
            payment_type VARCHAR(50),
            payment_installments INT,
            payment_value DECIMAL(10,2)
        )
        """
        cursor.execute(create_table_query)
        print(f"Table {table_name} created/verified successfully.")
        
        # Read CSV file using pandas
        df = pd.read_csv(csv_file_path)
        print(f"Loaded {len(df)} rows from CSV file.")
        
        # Insert data row by row
        insert_query = f"""
        INSERT INTO {table_name} 
        (order_id, payment_sequential, payment_type, payment_installments, payment_value)
        VALUES (%s, %s, %s, %s, %s)
        """
        
        # Convert DataFrame to list of tuples for insertion
        data_to_insert = []
        for index, row in df.iterrows():
            data_to_insert.append((
                row['order_id'],
                int(row['payment_sequential']),
                row['payment_type'],
                int(row['payment_installments']),
                float(row['payment_value'])
            ))
        
        # Execute batch insert
        cursor.executemany(insert_query, data_to_insert)
        connection.commit()
        
        print(f"Data loaded successfully into {table_name} table.")
        print(f"Inserted {cursor.rowcount} rows.")
        
except Error as e:
    print("Error while connecting to MySQL:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

C:\Users\visio\AppData\Local\Temp\ipykernel_37396\2360768104.py:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  print("Connected to MySQL Server", connection.get_server_info())


Connected to MySQL Server 8.0.36-28
Table order_payments created/verified successfully.
Loaded 103886 rows from CSV file.
Table order_payments created/verified successfully.
Loaded 103886 rows from CSV file.
Data loaded successfully into order_payments table.
Inserted 103886 rows.
MySQL connection is closed
Data loaded successfully into order_payments table.
Inserted 103886 rows.
MySQL connection is closed


Connecting to mongodb

In [ ]:
# Import MongoDB libraries
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, ServerSelectionTimeoutError
import json

In [ ]:
# MongoDB connection configuration


# For local MongoDB instance
mongodb_uri = "mongodb://localhost:27017/"
database_name = "olist_ecommerce"
collection_name = "order_payments"

In [ ]:
# Function to connect to MongoDB
def connect_to_mongodb(uri, db_name):
    try:
        # Create MongoDB client
        client = MongoClient(uri, serverSelectionTimeoutMS=5000)
        
        # Test the connection
        client.admin.command('ping')
        print("Successfully connected to MongoDB!")
        
        # Get database
        database = client[db_name]
        return client, database
        
    except ConnectionFailure:
        print("Failed to connect to MongoDB - Connection failure")
        return None, None
    except ServerSelectionTimeoutError:
        print("Failed to connect to MongoDB - Server selection timeout")
        print("Make sure MongoDB is running on your system")
        return None, None
    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        return None, None

In [ ]:
# Function to upload CSV data to MongoDB
def upload_csv_to_mongodb(csv_file_path, database, collection_name):
    try:
        # Read CSV file
        df = pd.read_csv(csv_file_path)
        print(f"Loaded {len(df)} rows from {csv_file_path}")
        
        # Convert DataFrame to list of dictionaries
        records = df.to_dict('records')
        
        # Get collection
        collection = database[collection_name]
        
        # Insert data
        result = collection.insert_many(records)
        print(f"Successfully inserted {len(result.inserted_ids)} documents into '{collection_name}' collection")
        
        return True
        
    except Exception as e:
        print(f"Error uploading data to MongoDB: {e}")
        return False

In [ ]:
# Connect to MongoDB and upload data
client, database = connect_to_mongodb(mongodb_uri, database_name)

if client and database:
    # Upload order payments data
    csv_file = "Data/olist_order_payments_dataset.csv"
    success = upload_csv_to_mongodb(csv_file, database, collection_name)
    
    if success:
        # Verify the upload by checking document count
        collection = database[collection_name]
        doc_count = collection.count_documents({})
        print(f"Total documents in '{collection_name}' collection: {doc_count}")
        
        # Show a sample document
        sample_doc = collection.find_one()
        print("\nSample document:")
        print(json.dumps(sample_doc, indent=2, default=str))
    
    # Close the connection
    client.close()
    print("\nMongoDB connection closed")
else:
    print("Failed to connect to MongoDB. Please check your connection settings.")

In [ ]:
# Upload all CSV files to MongoDB
csv_files = [
    ("Data/olist_customers_dataset.csv", "customers"),
    ("Data/olist_geolocation_dataset.csv", "geolocation"),
    ("Data/olist_order_items_dataset.csv", "order_items"),
    ("Data/olist_order_payments_dataset.csv", "order_payments"),
    ("Data/olist_order_reviews_dataset.csv", "order_reviews"),
    ("Data/olist_orders_dataset.csv", "orders"),
    ("Data/olist_products_dataset.csv", "products"),
    ("Data/olist_sellers_dataset.csv", "sellers"),
    ("Data/product_category_name_translation.csv", "product_categories")
]

# Connect to MongoDB
client, database = connect_to_mongodb(mongodb_uri, database_name)

if client and database:
    print("Uploading all CSV files to MongoDB...\n")
    
    for csv_file, collection_name in csv_files:
        print(f"Uploading {csv_file} to collection '{collection_name}'...")
        success = upload_csv_to_mongodb(csv_file, database, collection_name)
        
        if success:
            collection = database[collection_name]
            doc_count = collection.count_documents({})
            print(f"Collection '{collection_name}' now has {doc_count} documents\n")
        else:
            print(f"Failed to upload {csv_file}\n")
    
    print("Upload process completed!")
    
    # Show all collections in the database
    collections = database.list_collection_names()
    print(f"\nCollections in database '{database_name}':")
    for col in collections:
        count = database[col].count_documents({})
        print(f"  - {col}: {count} documents")
    
    # Close the connection
    client.close()
    print("\nMongoDB connection closed")
else:
    print("Failed to connect to MongoDB. Please check your connection settings.")